In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

In [2]:
gpu_id = 0  # Change this to the index of the desired GPU
torch.cuda.set_device(gpu_id)

In [3]:
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
num_classes = len(categories)
class CustomMobileNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomMobileNet, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained=True)
        num_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(1280,512,kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(512, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc6 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 512)
        self.dropout3 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(512, 64)
        self.dropout4 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc6(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.dropout4(x)
        x = self.fc4(x)
        x = torch.relu(x)
        x = self.fc5(x)
        return x

In [4]:
model_mbnet = CustomMobileNet(num_classes)
model_mbnet.load_state_dict(torch.load('model_torch.pth'))
model_mbnet.eval()

/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomMobileNet(
  (base_model): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_

In [5]:
categories_g = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
num_classes_g = len(categories_g)
class CustomGooglenet(nn.Module):
    def __init__(self, num_classes):
        super(CustomGooglenet, self).__init__()
        googlenet = models.googlenet(pretrained='googlenet')

        # Freeze all layers by default
        for param in googlenet.parameters():
            param.requires_grad = False

        for param in googlenet.inception5a.parameters():
            param.requires_grad = True

        for param in googlenet.inception5b.parameters():
            param.requires_grad = True

        for param in googlenet.fc.parameters():
            param.requires_grad = True

        self.googlenet = googlenet

        self.head = nn.Sequential(
            nn.Linear(1000, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048,2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            # nn.Linear(1024,1024),
            # nn.ReLU(),
            # nn.Dropout(0.3)
            # nn.Linear(4096, num_classes)
        )
        self.cnn1 = nn.Conv2d(2048, 1024, kernel_size = 3, padding = 1)
        self.dropout = nn.Dropout(0.5)
        self.cnn2 = nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1)
        self.dropout1 = nn.Dropout(0.4)
        self.cnn3 = nn.Conv2d(1024,512, kernel_size = 3, padding = 1)
        self.dropout2 = nn.Dropout(0.3)
        self.cnnl = nn.Conv2d(512, num_classes, kernel_size=3,padding=1)

    def forward(self, x):
        x = self.googlenet(x)
        x = torch.flatten(x, 1)
        x = self.head(x)
        x = x.view(x.size(0), -1,1,1)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.cnn3(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.cnnl(x)
        x = torch.flatten(x,1)
        return x

In [6]:
model_googlenet = CustomGooglenet(num_classes_g)
model_googlenet.load_state_dict(torch.load('model_torch_googlenet.pth'))
model_googlenet.eval()

/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomGooglenet(
  (googlenet): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     

In [7]:
class CustomMobileNet(nn.Module):
    def __init__(self, num_classes,fine_tune = False):
        super(CustomMobileNet, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained='imagenet')

        if not fine_tune:
            # Freeze all layers except the last few
            for param in self.base_model.parameters():
                param.requires_grad = False

            # Identify the last few layers of the base model and set requires_grad to True
            for param in self.base_model.features[-7:].parameters():
                param.requires_grad = True
        
        num_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(1280,2048,kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(2048, 2048)
        self.dropout2 = nn.Dropout(0.6)
        self.fc6 = nn.Linear(2048, 1024)
        self.dropout5 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 1024)
        self.dropout3 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1024, 512)
        self.dropout4 = nn.Dropout(0.4)
        # self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc6(x)
        x = torch.relu(x)
        x = self.dropout5(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.dropout4(x)
        # x = self.fc4(x)
        # x = torch.relu(x)
        x = self.fc5(x)
        return x

In [8]:
class CustomGooglenet_1(nn.Module):
    def __init__(self, num_classes):
        super(CustomGooglenet_1, self).__init__()
        googlenet = models.googlenet(pretrained=models.GoogLeNet_Weights.IMAGENET1K_V1)
        #googlenet = models.googlenet(pretrained='googlenet')


        # Freeze all layers by default
        for param in googlenet.parameters():
            param.requires_grad = False

        for param in googlenet.inception5a.parameters():
            param.requires_grad = True

        for param in googlenet.inception5b.parameters():
            param.requires_grad = True

        self.googlenet = nn.Sequential(*list(googlenet.children())[:-1])

        self.cnnl = nn.Sequential(
            nn.Conv2d(1024,512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(512,512,kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Conv2d(512,128,kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.AdaptiveAvgPool2d((1, 1))
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024,512, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(512,512,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.Conv2d(512,128,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.AdaptiveAvgPool2d((1, 1))
        )
        #self.cnnl = nn.Conv2d(512, num_classes, kernel_size=3,padding=1)
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes),
            # nn.ReLU(),
            # nn.Dropout(0.6),
            # nn.Linear(2048,1024),
            # nn.ReLU(),
            #nn.Linear(1024,1024),
            #nn.ReLU(),
            #nn.Dropout(0.3)
            # nn.Linear(4096, num_classes)
            )

    def forward(self, x):
        x = self.googlenet(x)
        x = x.view(x.size(0), -1,1,1)
        x = self.cnnl(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        # x = self.cnnl(x)
        # x = torch.flatten(x,1)
        return x

In [9]:
model_gnet_1 = CustomGooglenet_1(num_classes)
model_gnet_1.load_state_dict(torch.load('model_torch_googlenet_2.pth'))
model_gnet_1.eval()

CustomGooglenet_1(
  (googlenet): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (5): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.00

In [10]:
class CustomMobileNet_1(nn.Module):
    def __init__(self, num_classes,fine_tune = False):
        super(CustomMobileNet_1, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained='imagenet')

        if not fine_tune:
            # Freeze all layers except the last few
            for param in self.base_model.parameters():
                param.requires_grad = False

            # Identify the last few layers of the base model and set requires_grad to True
            for param in self.base_model.features[-3:].parameters():
                param.requires_grad = True
                
        num_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(1280,1024,kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.5)
        # self.cnn2 = nn.Conv2d(1024, 1024,kernel_size = 3, padding = 1)
        # self.dropout2 = nn.Dropout(0.6)
        self.cnn3 = nn.Conv2d(1024, 512,kernel_size = 3, padding = 1)
        self.dropout3 = nn.Dropout(0.5)
        # self.cnn4 = nn.Conv2d(512, 512,kernel_size = 3, padding = 1)
        # self.dropout4 = nn.Dropout(0.5)
        self.cnn5 = nn.Conv2d(512, 128,kernel_size = 3, padding = 1)
        self.dropout5 = nn.Dropout(0.4)
        self.avgpooling = nn.AdaptiveAvgPool2d((1, 1))
        self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])
        self.fc1 = nn.Linear(128, 64)
        self.dropout6 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(64,32)
        self.dropout7 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.base_model(x)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        # x = self.cnn2(x)
        # x = torch.relu(x)
        # x = self.dropout2(x)
        x = self.cnn3(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        # x = self.cnn4(x)
        # x = torch.relu(x)
        # x = self.dropout4(x)
        x = self.cnn5(x)
        x = torch.relu(x)
        x = self.dropout5(x)
        x = self.avgpooling(x)
        #x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout6(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout7(x)
        x = self.fc3(x)
        return x

In [11]:
model_mbnet_1 = CustomMobileNet_1(num_classes)
model_mbnet_1.load_state_dict(torch.load('model_torch_mobilenet_1.pth'))
model_mbnet_1.eval()

CustomMobileNet_1(
  (base_model): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(

In [12]:
class CustomMobileNet_2(nn.Module):
    def __init__(self, num_classes,fine_tune = False):
        super(CustomMobileNet_2, self).__init__()
        self.base_model = models.mobilenet_v3_large(pretrained=models.MobileNet_V3_Large_Weights.IMAGENET1K_V2)

        if not fine_tune:
            # Freeze all layers except the last few
            for param in self.base_model.parameters():
                param.requires_grad = False

            # Identify the last few layers of the base model and set requires_grad to True
            for param in self.base_model.features[-3:].parameters():
                param.requires_grad = True
                
        num_features = self.base_model.classifier[-1].in_features
        self.base_model.classifier[-1] = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(960,1024,kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(1024)
        self.dropout1 = nn.Dropout(0.5)
        # self.cnn2 = nn.Conv2d(1024, 1024,kernel_size = 3, padding = 1)
        # self.dropout2 = nn.Dropout(0.6)
        self.cnn3 = nn.Conv2d(1024, 512,kernel_size = 3, padding = 1)
        self.dropout3 = nn.Dropout(0.5)
        self.bn2 = nn.BatchNorm2d(512)
        # self.cnn4 = nn.Conv2d(512, 512,kernel_size = 3, padding = 1)
        # self.dropout4 = nn.Dropout(0.5)
        self.cnn5 = nn.Conv2d(512, 128,kernel_size = 3, padding = 1)
        self.dropout5 = nn.Dropout(0.4)
        self.bn3 = nn.BatchNorm2d(128)
        self.avgpooling = nn.AdaptiveAvgPool2d((1, 1))
        self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])
        self.fc1 = nn.Linear(128, 64)
        self.dropout6 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(64,32)
        self.dropout7 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.base_model(x)
        x = self.cnn1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        # x = self.cnn2(x)
        # x = torch.relu(x)
        # x = self.dropout2(x)
        x = self.cnn3(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        # x = self.cnn4(x)
        # x = torch.relu(x)
        # x = self.dropout4(x)
        x = self.cnn5(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout5(x)
        x = self.avgpooling(x)
        #x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout6(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout7(x)
        x = self.fc3(x)
        return x

In [13]:
model_mbnet_2 = CustomMobileNet_2(num_classes)
model_mbnet_2.load_state_dict(torch.load('model_torch_mobilenet_2.pth'))
model_mbnet_2.eval()

/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomMobileNet_2(
  (base_model): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivati

In [43]:
from PIL import Image

image_path = 'img_l_1.jpg'
image = Image.open(image_path).convert("RGB")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
input_image = transform(image).unsqueeze(0)

In [44]:
with torch.no_grad():
    output = model_mbnet(input_image)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = categories[predicted_class_index]

In [45]:
predicted_class_index

14

In [46]:
predicted_class_name

'Laryngoscopes'

In [47]:
with torch.no_grad():
    output = model_googlenet(input_image)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = categories_g[predicted_class_index]

In [48]:
predicted_class_name

'mayfield_clamp'

In [49]:
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
with torch.no_grad():
    output = model_gnet_1(input_image)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = categories[predicted_class_index]
print(predicted_class_name)

C_arm


In [50]:
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
with torch.no_grad():
    output = model_mbnet_1(input_image)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = categories[predicted_class_index]
print(predicted_class_name)

Electrocautery_devices


In [51]:
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
with torch.no_grad():
    output = model_mbnet_2(input_image)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = categories[predicted_class_index]
print(predicted_class_index)
print(predicted_class_name)

5
Curved_Mayo_Scissor


In [33]:
import random

In [34]:
DATA_DIR = 'RediMinds/Data_Cleaned/'

def predict_image(model, image):
    with torch.no_grad():
        image = transform(image).unsqueeze(0)
        output = model_mbnet_2(image)
        _, predicted_class = torch.max(output, 1)
        class_index = predicted_class.item()
        class_label = categories[class_index]
        return class_label

In [114]:
count = 0
folder_names = os.listdir(DATA_DIR)
for folder_name in folder_names:
    folder_path = os.path.join(DATA_DIR, folder_name)
    image_files = os.listdir(folder_path)
    if len(image_files) > 0:
        random_image_name = random.choice(image_files)
        print(random_image_name)
        image_path = os.path.join(folder_path, random_image_name)
        image = Image.open(image_path).convert('RGB')
        prediction = predict_image(model_mbnet_2, image)
        print("\n---------------------------------------------------------------------------\n")
        print(f"Folder: {folder_name} | Prediction: {prediction}")
        print("\n---------------------------------------------------------------------------\n")
        if(folder_name == prediction):
            count += 1
print("\n\n",count)

41cbMk3H8nL._AC_UL800_QL65_.jpeg

---------------------------------------------------------------------------

Folder: Bone_saws | Prediction: forceps

---------------------------------------------------------------------------

images298.jpeg

---------------------------------------------------------------------------

Folder: colonoscope | Prediction: colonoscope

---------------------------------------------------------------------------

pinca70_resized_.jpeg

---------------------------------------------------------------------------

Folder: Straight_Dissection_Clamp | Prediction: Straight_Dissection_Clamp

---------------------------------------------------------------------------

images411.jpeg

---------------------------------------------------------------------------

Folder: baby_incubator | Prediction: baby_incubator

---------------------------------------------------------------------------

Valley-Lab-Cautery-2-300x298.jpeg

--------------------------------------------